In [6]:
import keras
import numpy as np
from pond.tensor import NativeTensor, PublicEncodedTensor, PrivateEncodedTensor
from pond.nn import Dense, ReluExact, Relu, Reveal, CrossEntropy, SoftmaxStable, Sequential, DataLoader, Conv2D, \
                                                                                  AveragePooling2D, Flatten, ConvAveragePooling2D
from keras.utils import to_categorical

# set errors error behaviour for overflow/underflow
_ = np.seterr(over='raise')
_ = np.seterr(under='raise')
_ = np.seterr(invalid='raise')

## Read data

In [7]:
# read data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train[:,np.newaxis,:,:] / 255.0
x_test = x_test[:,np.newaxis,:,:] / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

batch_size = 128
input_shape = [batch_size] + list(x_train.shape[1:])

## Define 2 convnets

In [10]:
convnet_shallow = Sequential([
    Conv2D((3, 3, 1, 32), strides=1, padding=1, filter_init=lambda shp: np.random.uniform(low=-0.1, high=0.1, size=shp)),
    AveragePooling2D(pool_size=(2, 2)),
    Relu(order=3),
    Flatten(),
    Dense(10, 6272),
    Reveal(),
    SoftmaxStable()
])


# deep convnet architecture. Which is not used b
convnet_deep = Sequential([
    Conv2D((3, 3, 1, 32), strides=1, padding=1, filter_init=lambda shp: np.random.uniform(low=-0.14, high=0.14, size=shp)),
    AveragePooling2D(pool_size=(2, 2)),
    Relu(),
    Conv2D((3, 3, 32, 32), strides=1, padding=1, filter_init=lambda shp: np.random.uniform(low=-0.1, high=0.1, size=shp)),
    AveragePooling2D(pool_size=(2, 2)),
    Relu(),
    Flatten(),
    Dense(10, 1568),
    Reveal(),
    SoftmaxStable()
])


In [12]:
tensortype = NativeTensor
convnet_shallow.initialize(initializer=tensortype, input_shape=input_shape)
convnet_shallow.fit(
    x_train=DataLoader(x_train, wrapper=tensortype),
    y_train=DataLoader(y_train, wrapper=tensortype),
    x_valid=DataLoader(x_test, wrapper=tensortype),
    y_valid=DataLoader(y_test, wrapper=tensortype),
    loss=CrossEntropy(),
    epochs=1,
    batch_size=batch_size,
    verbose=1,
    learning_rate=0.01
)


2018-05-14 13:34:48.812524 Epoch 1/1
2560/60032 [=>.............................] - ETA: 0:05:25 - train_loss: 2.30088 - train_acc 0.08594

KeyboardInterrupt: 

## PublicEncodedTensor
Train the same network on PublicEncodedTensor, this network **does not** use SPDZ, but works on the 128 bit integer encoding of real numbers (which slows down 100x). Therefore, we only train on the first two batches.

In [15]:
# train on small set
x_train = x_train[:256]
y_train = y_train[:256]
x_test = x_test[:256]
y_test = y_test[:256]

tensortype = PublicEncodedTensor
convnet_shallow.initialize(initializer=tensortype, input_shape=input_shape)
convnet_shallow.fit(
    x_train=DataLoader(x_train, wrapper=tensortype),
    y_train=DataLoader(y_train, wrapper=tensortype),
    x_valid=DataLoader(x_test, wrapper=tensortype),
    y_valid=DataLoader(y_test, wrapper=tensortype),
    loss=CrossEntropy(),
    epochs=1,
    batch_size=batch_size,
    verbose=1,
    learning_rate=0.01
)


2018-05-14 13:40:39.128344 Epoch 1/1
256/256 [===============>...............] - ETA: 0:01:20 - train_loss: 2.29954 - train_acc 0.10156 - val_loss 2.29064 - val_acc 0.13672


## PrivateEncodedTensor
Train the same network on PublicEncodedTensor, this network **does** use SPDZ, and is even slower. Here, we also train on the first two batches. However, if you have time you can run the network for a full epoch and reach ~86% accuracy

In [ ]:
# train on small set
x_train = x_train[:256]
y_train = y_train[:256]
x_test = x_test[:256]
y_test = y_test[:256]

tensortype = PrivateEncodedTensor
convnet_shallow.initialize(initializer=tensortype, input_shape=input_shape)
convnet_shallow.fit(
    x_train=DataLoader(x_train, wrapper=tensortype),
    y_train=DataLoader(y_train, wrapper=tensortype),
    x_valid=DataLoader(x_test, wrapper=tensortype),
    y_valid=DataLoader(y_test, wrapper=tensortype),
    loss=CrossEntropy(),
    epochs=1,
    batch_size=batch_size,
    verbose=1,
    learning_rate=0.01
)
